In [1]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
### HAMILTONIAN start
Molecule = 'H2O'
geometry = None
num_shots = 10000
basis = 'sto-3g'


### Get Hamiltonian
Hamilt = Hamiltonian(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation='JW')
### HAMILTONIAN end

#####################################

Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(QubitHamiltonian)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'
anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)
anti_commuting_sets

Building Graph Edges: 100%|##########| 1390/1390 [01:24<00:00, 16.40it/s] 


{0: [(-46.465600783681765+0j) []],
 1: [(0.22003977334376174+0j) [Z8 Z9]],
 2: [(1.369285279309879+0j) [Z9],
  (-0.006509359453068324+0j) [X0 X1 Y8 Y9],
  (0.008126859402904038+0j) [X0 Z1 Z2 X3 Y8 Y9],
  (0.008126859402904038+0j) [X1 Y2 Y8 X9],
  (0.0033479000466714367+0j) [X0 Z1 Z2 Z3 Z4 Z5 Z6 X7 Y8 Y9],
  (0.003347900046671437+0j) [X1 Z2 Z3 Z4 Z5 Y6 Y8 X9],
  (0.0038801488134229643+0j) [X1 Z2 Z3 Z4 Z5 Z6 Z7 X8 Y9 Y10],
  (-0.0038801488134229648+0j) [X0 Z1 Z2 Z3 Z4 Z5 Z6 Z7 X8 X9 Z10 X11]],
 3: [(1.3692852793098786+0j) [Z8],
  (0.006509359453068324+0j) [Y0 X1 X8 Y9],
  (0.008126859402904038+0j) [Y1 X2 X8 Y9],
  (0.003347900046671437+0j) [Y1 Z2 Z3 Z4 Z5 X6 X8 Y9],
  (-0.03620499227819434+0j) [X2 X3 Y8 Y9],
  (-0.011728119903768002+0j) [X2 Z3 Z4 Z5 Z6 X7 Y8 Y9],
  (0.014616824405077406+0j) [X2 Z3 Z4 Z5 Z6 Z7 X8 X9 Z10 X11],
  (0.0038801488134229643+0j) [Y1 Z2 Z3 Z4 Z5 Z6 Z7 Y8 X9 X10]],
 4: [(12.412562749529524+0j) [Z1],
  (-0.1251015480257414+0j) [X1 Z2 X3],
  (-0.04260477126459847+0j)

In [2]:
from quchem.Ansatz_Generator_Functions import *

##
NMO_basis, NOON_spins_combined = Hamilt.Get_NOON()
##
Hamilt.Get_CCSD_Amplitudes()



ansatz_obj = Ansatz(Hamilt.molecule.n_electrons, Hamilt.molecule.n_qubits)
reduced_Sec_Quant_CC_ops_ia, reduced_Sec_Quant_CC_ops_ijab, reduced_theta_parameters_ia, reduced_theta_parameters_ijab =ansatz_obj.Remove_NOON_terms(
    NMO_basis=NMO_basis,
    occ_threshold= 0.999,
    unocc_threshold=1e-4,
    indices_to_remove_list_manual=None, 
    single_cc_amplitudes=Hamilt.molecule.single_cc_amplitudes,
    double_cc_amplitudes=Hamilt.molecule.double_cc_amplitudes,
    singles_hamiltonian=Hamilt.singles_hamiltonian,
    doubles_hamiltonian=Hamilt.doubles_hamiltonian,
    tol_filter_small_terms=None)
reduced_Sec_Quant_CC_ops_ijab

[-1.0 [0^ 1^ 10 11] +
 1.0 [11^ 10^ 1 0],
 -1.0 [0^ 3^ 10 11] +
 1.0 [11^ 10^ 3 0],
 -1.0 [0^ 7^ 10 11] +
 1.0 [11^ 10^ 7 0],
 -1.0 [2^ 3^ 10 11] +
 1.0 [11^ 10^ 3 2],
 -1.0 [2^ 7^ 10 11] +
 1.0 [11^ 10^ 7 2],
 -1.0 [4^ 7^ 10 11] +
 1.0 [11^ 10^ 7 4],
 -1.0 [6^ 7^ 10 11] +
 1.0 [11^ 10^ 7 6]]

In [3]:
ia_terms, ijab_terms, ia_theta, ijab_theta = ansatz_obj.Get_ia_and_ijab_terms()

print('REDUCTION')
print('ia_terms', len(ia_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ia))
print('ijab_terms', len(ijab_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ijab))

REDUCTION
ia_terms 20 TO 8
ijab_terms 65 TO 7


In [4]:
Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab = ansatz_obj.UCCSD_single_trotter_step(reduced_Sec_Quant_CC_ops_ia,
                                                                                                                    reduced_Sec_Quant_CC_ops_ijab)

full_ansatz_Q_Circ = Ansatz_Circuit(Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab,
             Hamilt.molecule.n_qubits, Hamilt.molecule.n_electrons)

ansatz_cirq_circuit = full_ansatz_Q_Circ.Get_Full_HF_UCCSD_QC(reduced_theta_parameters_ia, reduced_theta_parameters_ijab)

print(ansatz_cirq_circuit.to_text_diagram(transpose=True)) 

  0         1         2         3         4         5 6         7         8         9 10          11
  │         │         │         │         │         │ │         │         │         │ │           │
  X         X         X         X         X         X X         X         X         X H           H
  │         │         │         │         │         │ │         │         │         │ │           │
  Rx(0.5π)  │         │         │         │         │ │         │         │         │ │           │
  │         │         │         │         │         │ │         │         │         │ │           │
  @─────────X         │         │         │         │ │         │         │         │ │           │
  │         │         │         │         │         │ │         │         │         │ │           │
  │         @─────────X         │         │         │ │         │         │         │ │           │
  │         │         │         │         │         │ │         │         │         │ │           │

In [5]:

simulator = cirq.Simulator()
result = simulator.compute_amplitudes(ansatz_cirq_circuit, bitstrings=[i for i in range(2 ** len(ansatz_cirq_circuit.all_qubits()))])
result=np.around(result, 5)
print(result.reshape([(2 ** len(ansatz_cirq_circuit.all_qubits())), 1]))

[[ 0.+0.j]
 [ 0.+0.j]
 [-0.+0.j]
 ...
 [-0.+0.j]
 [ 0.+0.j]
 [-0.-0.j]]


In [6]:
from quchem.LCU_method import *

In [7]:
from quchem.Simulating_Quantum_Circuit import *
R_uncorrected, Pn, gamma_l = Get_R_linear_combination(anti_commuting_sets[10], 1)
R_corrected_Op_list, R_corr_list, ancilla_amplitudes, l1 = absorb_complex_phases(R_uncorrected)

full_circ = Full_Q_Circuit(Pn, R_corrected_Op_list, R_corr_list, ancilla_amplitudes, Hamilt.molecule.n_qubits, ansatz_cirq_circuit)
output_bin_dict = Get_binary_dict_project(full_circ, Pn, 100000, Hamilt.molecule.n_qubits, ancilla_amplitudes, l1)
expectation_value_by_parity(output_bin_dict) * gamma_l 

(-1.3023459801097672+0j)

In [8]:
n_shots=10


testing =  VQE_Experiment_LCU_UP(anti_commuting_sets,
                 ansatz_cirq_circuit,
                 n_shots,
                 Hamilt.molecule.n_qubits,
                 N_indices_dict=None)
testing.Calc_Energy()

(-74.5038644322939+0j)

In [9]:
Hamilt.molecule.fci_energy

array(-75.01553019)

In [15]:
n_shots=10

def GIVE_ENERGY(theta_ia_theta_jab_list):
    theta_ia = theta_ia_theta_jab_list[:len(reduced_theta_parameters_ia)]
    theta_ijab = theta_ia_theta_jab_list[len(reduced_theta_parameters_ia):]
   
    ansatz_cirq_circuit = full_ansatz_Q_Circ.Get_Full_HF_UCCSD_QC(theta_ia, theta_ijab)

    VQE_exp_LCU =  VQE_Experiment_LCU_UP(anti_commuting_sets,
                     ansatz_cirq_circuit,
                     n_shots,
                     Hamilt.molecule.n_qubits,
                     N_indices_dict=None)#{7:0, 8:1, 9:0, 10:1})
    return VQE_exp_LCU.Calc_Energy().real


In [ ]:
### optimizer
from quchem.Scipy_Optimizer import *
THETA_params=[0 for _ in range(len(reduced_theta_parameters_ia)+len(reduced_theta_parameters_ijab))]

GG = Optimizer(GIVE_ENERGY, THETA_params, 'Nelder-Mead', store_values=True, display_iter_steps=True,
               tol=1e-5,
               display_convergence_message=True)
GG.get_env(50)
GG.plot_convergence()
plt.show()

### Tensorflow

In [11]:
from quchem.TensorFlow_Opt import *

**gradient is given by**

https://arxiv.org/pdf/1906.08728.pdf

$$\frac{\partial O(\theta)}{\partial \theta}=\left\langle\overrightarrow{0}\left|\hat{U}^{\dagger} \hat{R}_{y}^{C \dagger}(\theta+\pi / 4) \hat{V}^{\dagger} \hat{O} \hat{V} \hat{R}_{y}^{C}(\theta+\pi / 4) \hat{U}\right| \overrightarrow{0}\right\rangle$
$-\left\langle\overrightarrow{0}\left|\hat{U}^{\dagger} \hat{R}_{y}^{C \dagger}(\theta-\pi / 4) \hat{V}^{\dagger} \hat{O} \hat{V} \hat{R}_{y}^{C}(\theta-\pi / 4) \hat{U}\right| \overrightarrow{0}\right\rangle$$

$$\frac{\partial O(\theta)}{\partial \theta} =O(\theta+\pi / 4)-O(\theta-\pi / 4)$$

In [16]:
def calc_gradient(theta_ia_theta_jab_list):
    
    grad_list=[]
    
    for index, theta in enumerate(theta_ia_theta_jab_list):
        
        new_theta_list = theta_ia_theta_jab_list.copy()
        new_theta_list[index] = theta + np.pi/4
        
        Obs_PLUS = GIVE_ENERGY(new_theta_list)
        
        
        new_theta_list[index] = theta - np.pi/4
        Obs_MINUS = GIVE_ENERGY(new_theta_list)
        
        gradient = Obs_PLUS - Obs_MINUS
        
        grad_list.append((gradient, theta))
        

    return grad_list

In [ ]:
import random

X0 = [random.uniform(0, 2*np.pi) for _ in range(len(reduced_Sec_Quant_CC_ops_ia) + len(reduced_Sec_Quant_CC_ops_ijab))]

GG = Tensor_Flow_Optimizer(GIVE_ENERGY, X0, 'Adam', calc_gradient, learning_rate=0.1, beta1=0.9,
                            beta2=0.999, store_values=True, display_iter_steps=True)
GG.optimize(50)
GG.plot_convergence()

starting input: [3.454913082128326, 0.6566565610793452, 1.3812442297416228, 3.5151030406432544, 3.9732553692412536, 0.32586480390413514, 3.020289413061396, 3.5465997397314464, 3.1030150062726634, 0.5952527107609289, 4.388437721823473, 1.6293691516329196, 3.264309344839584, 4.163063050062637, 5.973023508217085] obj funct out: -66.83882279424472
